Backpropagation from hand, part 2: Tensors.

Old code:

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build dataset
block_size = 3

def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


New code:

In [5]:
# comparison function for gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # character embedding vector dimension
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5) 
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # biases would normally be all 0

# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1 # biases would normally be all 0

# Batchnorm parameters
bngain = torch.randn((1, n_hidden) , generator=g) * 0.1 + 1.00
bnbias = torch.randn((1, n_hidden), generator=g) * 0.1
# batchnorm removes need for bias b1 in first layer

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch

In [8]:
# forward pass written down to individual steps

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
hprebn = embcat @ W1 + b1

# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # Bessel correction /(n-1), not /n
bnvar_inv = (bnvar + 1e-5)**-0.5 # 1/std, or 1/sqrt(var + eps)
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity in hidden layer
h = torch.tanh(hpreact) 

# Linear layer 2, output layer
logits = h @ W2 + b2

# Cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # for numeric stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # he comments that 1.0/sum is not exact? "to be bit exact"
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, 
         norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.5571, grad_fn=<NegBackward0>)

In [9]:
emb.shape

torch.Size([32, 3, 10])

In [10]:
embcat.view(32, 3, 10).shape

torch.Size([32, 3, 10])

In [11]:
(hprebn @ W1.T).shape

torch.Size([32, 30])

#### Exercise 1 

In [12]:
# his dsomething means dLoss / dsomething
# use dL / dsomething = dL / donestepbefore * donestepbefore / dsomething
# the first term you already have calculated, the second term you can calculate from the expression for "onestepbefore" as a function of "something"
# multiple dependencies are additive
# sometimes the solution could be guessed by investigating the shapes of the involved tensors

dlogprobs = torch.zeros((n, vocab_size)) # he used torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = - 1.0/n

dprobs = dlogprobs / probs 

dcounts_sum_inv =  (counts @ dprobs.T).diag().view(32, 1) # super strange
# his solution: (counts * dprobs).sum(1, keepdim=True)

dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

dcounts = torch.ones((32, 27)) * (dcounts_sum + counts_sum_inv * dprobs)
# for the first term he used torch.ones_like(counts)
# he uses += for both partial dcounts results
 
dnorm_logits = dcounts * counts

dlogit_maxes = -dnorm_logits.sum(1, keepdims=True) # would assume derivative is exactly 0

t = torch.zeros(logits.shape)
# the maximum appears for each entry, hence needs to be summed over the gradient for each entry
t[range(t.shape[0]), logits.argmax(dim=1)] =  dlogit_maxes.view(-1) 
dlogits = dnorm_logits + t
# his comments:
#   - logits.max(1).indices has the indices (that I got with argmax)
#   - F.one_hot to get a matrix with 0, and 1's at the right places

dh = dlogits @ W2.T # guessed based on matrix dimension matching

# I forgot the b and W2, because I took all variables that he assigned "retain_grad" to
# why not retain_grad to b2 and W2 etc?

dhpreact = dh * (1.0 - h**2)

dbnraw = dhpreact * bngain.repeat(batch_size, 1)

dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)

dbnvar = dbnvar_inv * -0.5*(bnvar + 1e-5)**-1.5

dbndiff2 = dbnvar.repeat(batch_size, 1) / (n - 1) # I don't see the mistake here, max difference to true result is 1.45e-11
dbndiff2v2 = dbnvar.repeat(batch_size, 1) * (n - 1)**-1 # this works

dbndiff = dbndiff2 * 2*bndiff + dbnraw * bnvar_inv

dbnmeani = -dbndiff.sum(0, keepdim=True)

dhprebn = dbndiff + dbnmeani.repeat(batch_size, 1)/n

dembcat = dhprebn @ W1.T

demb = dembcat.view(32, 3, 10)

# forgot:
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnbias = dhpreact.sum(0, keepdim=True)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs) 
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv) 
cmp('counts_sum', dcounts_sum, counts_sum) 
cmp('counts', dcounts, counts) 
cmp('norm_logits', dnorm_logits, norm_logits) 
cmp('logit_maxes', dlogit_maxes, logit_maxes) 
cmp('logits', dlogits, logits) 
cmp('h', dh, h) 
cmp('hpreact', dhpreact, hpreact) 
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv) 
cmp('bnvar', dbnvar, bnvar) 
print("\nI don't see the mistake here, even when debugging element-by-element:")
cmp('bndiff2', dbndiff2, bndiff2)
print("\nThese are then consequentially also wrong:")
cmp('bndiff', dbndiff, bndiff) 
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn) 
cmp('embcat', dembcat, embcat) 
cmp('emb', demb, emb)

cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('C', dC, C)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)


print("\nIt is possible to obtain 0 difference by replacing /(n-1) by (n-1)**-1:")
cmp('bndiff2', dbndiff2v2, bndiff2)
print("All follow-on errors vanish then as well.")





logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bnvar_inv       | exact: True  | approximate: True  | maxdiff: 0.0
bnvar           | exact: True  | approximate: True  | maxdiff: 0.0

I don't see the mistake here, even when debugging element-by-element:
bndiff2         | exact: False | approximate: True  | maxd

#### Exercise 2

In [13]:
# backprop of cross-entropy loss
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.5571467876434326 diff: -2.384185791015625e-07


In [67]:
# manually derived
dlogits = (logits.softmax(1) - F.one_hot(Yb, num_classes=27))/32

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 5.122274160385132e-09


#### Exercise 3

In [68]:
# batchnorm layer
# forward pass in one go:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias


In [74]:
# backward pass
# calculate dhprebn given dhpreact
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # tried once, but then followed his solution


hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [75]:
# Exercise 4 copy everything together: left it out